In [ ]:
from paddleocr import PaddleOCR,draw_ocr
from PIL import Image
from IPython.display import display
from llama_index.llms.ollama import Ollama

In [ ]:
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log = False)
# EDIT ME: change the value of img_path to your file's location.
img_path = 'data/1996-388-035.pdf'
result = ocr.ocr(img_path, cls=True)
ocr_text = ''
for res in result:
    for line in res:
        ocr_text += (line[-1][0] + '\n')

print(ocr_text)

In [ ]:
from refined.data_types.base_types import Span, Entity
from refined.inference.processor import Refined

In [ ]:
refined = Refined.from_pretrained(model_name='wikipedia_model',
                                  entity_set='wikipedia',
                                  use_precomputed_descriptions=False,
                                  download_files=True,
                                  device='cpu')

In [ ]:
def construct_wikidata_url(id):
    return "https://www.wikidata.org/wiki/" + id
    
text = ocr_text
spans = refined.process_text(text)

named_entities = []
for span in spans:
    id = span.predicted_entity.wikidata_entity_id
    name = span.predicted_entity.wikipedia_entity_title

    if name is not None and id is not None:
        url = construct_wikidata_url(id)
        full_entity = (name_url)
        if full_entity not in named_entities:
            named_entities.append((name, url))
        



In [ ]:
description_prompt = ("The below text is taken from a political advertisement."
    f" Using neutral language, summarize its contents. '''{ocr_text}'''"
    )
# title prompt needs work
title_prompt =  (" Using neutral language, provide a short title for this political advertisement in less than 10 words: "
                f"'''{ocr_text}'''")

llm = Ollama(base_url="http://ollama:11434", model="tinyllama", request_timeout=180.0)
description = llm.complete(description_prompt)
title = llm.complete(title_prompt)
print(description)
print()
print(title)

In [ ]:
import json
record_dict = {"title": str(title),
               "description": str(description),
               "related_entities": named_entities}

with open("output_record.json", "w") as outfile: 
    json.dump(record_dict, outfile)

